# Workers and Queues

### Introduction

In the last lesson, we learned about setting up a deployment with prefect, and we saw how our deployment is associated with a particular flow, and that it schedules flow runs.

In this lesson, we'll look a little bit deeper at 

### Starting with our Flow

Ok, if you look at the current codebase, we have a flow that's defined in our `index.py` file, and the `get_data` function is our flow function.

```python
import requests
from prefect import flow, task

@task
def find_receipts(name):
    url = "https://data.texas.gov/resource/naix-2893.json"
    response = requests.get(url, params = {'taxpayer_name': name})
    return response.json()[:1]

@flow
def get_restaurants(url: str):
    receipts = find_receipts(url)
    return receipts

name = 'HONDURAS MAYA CAFE & BAR LLC'
print(get_restaurants(name))
```

```python
@task
def find_receipts(name):
    url = "https://data.texas.gov/resource/naix-2893.json"
    response = requests.get(url, params = {'taxpayer_name': name})
    return response.json()[:1]

@flow
def get_data(url: str):
    receipts = find_receipts(url)
    return receipts

name = 'HONDURAS MAYA CAFE & BAR LLC'
print(get_data(name))
```

### Creating a deployment

Now it's time to create a deployment, which will allow us to schedule our workflow.

To create a deployment, first make sure that your local prefect server is running.  Navigate to the prefect codebase and run the following.

```bash
prefect server start
```

And we can create our deployment file with the following.

```bash
prefect deployment build ./index.py:get_data -n get-restaurants -q test
```

Ok, so this is how to make sense of the above command:

* `prefect deployment build` is the standard command for building a deployment
* `./index.py:get_data` specifies the location of the flow function -- the file followed by the name of the function
* `-n get-restaurants-flow` specifies the deployment name.
* `-q test` is the work pool for the deployment. Where a work pool directs scheduled runs to an agent.

```
prefect deployment build ./index.py:get_data -n get-restaurants -q test
```

Look at the command above, can you identify what each component means?

Now put the line above into your terminal. It creates a new file in our codebase called `get_data-deployment.yaml`.  If you look through the new yaml file, you'll see some of our specifications, as well as other information that prefect filled in.

```yaml
###
### A complete description of a Prefect Deployment for flow 'get-data'
###
name: get-restaurants
description: null
version: 1f5a2d8c7f74279cc07fbc62a533b808
# The work queue that will handle this deployment's runs
work_queue_name: test
work_pool_name: null
tags: []
parameters: None
....
```

### Adding a schedule

Ok, so our CLI command filled in a lot of info for our deployment, but one key attribute that is still not filled in is a [schedule](https://docs.prefect.io/2.10.12/concepts/schedules/).  The schedule indicates how often we would like this deployment to be run.  So let's take a look at it.  We currently see that the `schedule` attribute in our deployment yaml is set to the following:

```yaml
schedule: null
```

To the following:

```yaml
schedule:
  interval: 60
```

This will specify to run the deployment every 60 seconds.

### Adding a parameter

We're almost ready to apply our deployment, but it turns out our flow will currently break.  If you look at our flow, you'll see that it takes an argument of the url.

```python
@flow
def get_data(url: str):
    receipts = find_receipts(url)
    return receipts
```

So we'll want our flow function `get_data` to get called with an argument for the url.  We can make sure it is by updating the deployment file's `parameters` attribute.

```yaml
parameters: {'url': 'HONDURAS MAYA CAFE & BAR LLC'}
```

This will invoke our flow function with a url value of the Honduras string.

### Applying the deployment

Ok, now that we've updated the schedule and the parameters, it's time to apply the deployment.  Let's do so with by issuing the following CLI command from the terminal:

```bash
prefect deployment apply get_data-deployment.yaml
```

In the terminal prefect will provide a link as to where to view the deployment.  So click on the link provided or copy and paste it into your browser.

<img src="./get-restaurants-flow.png">

The link should take you to the deployments dashboard in prefect.io.  

> If it doesn't, just go there by logging into `prefect.io`, going to your workspace and clicking on deployments.

<img src="./deployments.png">

And you can see information about the deployment if you then click on the name of the deployment.

> Eventually, we'll see it's unhealthy.

<img src="./deployments-work.png">

And if we want, we can even try to kick off a deployment-run by clicking the run function on the top right.

In [ ]:
prefect agent start --pool default-agent-pool --work-queue test

The issue is that we need to assign a work pool that will handle this deployment.

<img src="./workqueue.png">

Ok, so now our deployment is working.  And it's running our flow every minute.

Finally, if we would like to stop a deployment, we can do so by going to prefect cloud, clicking on deployments and clicking on the button over to the right so that it is no longer green.

<img src="./deployment-pause.png">

### Summary

In this lesson we saw how to schedule a flow with a deployment.  We did so by first creating our deployment yaml file.

```bash
prefect deployment build ./index.py:get_data -n get-restaurants -q test
```

And then we updated our deployment file's schedule and parameters values.

```yaml
schedule:
  interval: 60
```

```yaml
parameters: {'url': 'HONDURAS MAYA CAFE & BAR LLC'}
```

With this our flow will be run every 60 seconds, and the flow will be run with the url parameter passed through as an argument.

We then applied the deployment.

```bash
prefect deployment apply get_data-deployment.yaml
```

And finally assigned a work pool for the deployment.

```bash
prefect agent start --pool default-agent-pool --work-queue test
```